<i><b>Public AI</b></i>

# Optimizer (1) Momentum

### _Objective_


* Momentum 알고리즘의 원리인 지수이동평균에 대해 배워보겠습니다. <br>
* Gradient Descent 알고리즘의 개량 버전인 Momentum 알고리즘을 배워보겠습니다. <br>


In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.mplot3d import Axes3D
from tensorflow.keras.utils import get_file
from tqdm import tqdm

np.random.seed(30)
if "set_random_seed" in dir(tf.random):
    tf.random.set_random_seed(30)
else:
    tf.random.set_seed(30)    

<br><br>

# \[ 1. 지수이동평균 \]

----

* 지수 이동 평균이란, 시계열적 데이터에서 전체적인 추세를 확인하는 방법론입니다.
* 증권가의 차트를 분석할 때 많이 등장하는 것으로, 변화가 극심한 차트 정보에서, 큰 추세의 변화를 알아내는 방법론입니다.

### 예제 데이터) 2004.01~2018.05까지의 코스피 지수

In [ ]:
fpath = get_file("KOSPI_dataset.csv",
                 "https://s3.ap-northeast-2.amazonaws.com/pai-datasets/alai-deeplearning/KOSPI_dataset.csv")

df = pd.read_csv(fpath,index_col=0)
df.plot(y='DATA_VALUE',title="kospi")
plt.show()

<br>

## 1. 지수이동평균 (EMA, exponential Moving Average) 수식


* 지수이동평균의 수식은 아래와 같습니다.<br>
$
s_{n+1} = (1-\beta) x_n + \beta s_n\\
x_n: \mbox{최신 정보} \\
s_n: \mbox{과거 종합 정보} \\
\beta : \mbox{계수},0<\beta<1
$


* 위의 지수이동평균값($s_{n+1}$)의 재귀식을 풀어보면 아래와 같이 나옵니다.
$
s_{n+1} = (1-\beta) ( x_n + \beta x_{n-1}+\beta^2x_{n-2}+ \cdots + \beta^{n}x_{0})
$
* 과거의 정보일수록 가중치가 점점 작아지는 특성을 가집니다.


<br>

## 2. 지수이동 평균 계산하기

### (1) $\beta$가 0.9일 때

In [ ]:
beta = 0.9
ema = 0
ema_list = []

for x in df.DATA_VALUE:
    ema = (1-beta)*x + beta*ema
    ema_list.append(ema)

df['beta=0.9'] = ema_list    

In [ ]:
df.plot(y=['DATA_VALUE','beta=0.9'],title="kospi")
plt.show()

### (2) $\beta$가 0.5일 때

In [ ]:
beta = 0.5
ema = 0
ema_list = []

for x in df.DATA_VALUE:
    ema = (1-beta)*x + beta*ema
    ema_list.append(ema)

df['beta=0.5'] = ema_list    

In [ ]:
df.plot(y=['DATA_VALUE','beta=0.9','beta=0.5'],title="kospi")
plt.show()

beta 값이 작아질수록 좀 더 변화에 민감하게 움직인다는 것을 알 수 있습니다.

## 3. 편향 보정 수식

*  EMA 값을 구할 때, 초기 EMA 값이 너무 적게 계산됩니다.<br>
* 이 문제를 해결하기 위해 기본적으로 편향 보정 가중치를 곱해주게 됩니다.<br>
$
s_{n+1} =\frac{1}{1-\beta^{n+1}}((1-\beta) x_n + \beta s_n)
$

### (1)  $\beta$가 0.9일 때

In [ ]:
beta = 0.9
ema = 0
ema_list = []

for idx, x in enumerate(df.DATA_VALUE):
    ema = (1-beta)*x + beta*ema
    adjusted_ema = ema / (1-beta**(idx+1))
    ema_list.append(adjusted_ema)

df['beta=0.9'] = ema_list    

In [ ]:
df.plot(y=['DATA_VALUE','beta=0.9'],title="kospi")
plt.show()

### (2) $\beta$가 0.5일 때

In [ ]:
beta = 0.5
ema = 0
ema_list = []

for idx,x in enumerate(df.DATA_VALUE):
    ema = (1-beta)*x + beta*ema
    adjusted_ema = ema / (1-beta**(idx+1))
    ema_list.append(adjusted_ema)

df['beta=0.5'] = ema_list    

In [ ]:
df.plot(y=['DATA_VALUE','beta=0.9','beta=0.5'],title="kospi")
plt.show()

편향 보정을 통해, 초기에 값이 지나치게 작게 평가되는 문제를 방지하였습니다.

<br><br>

# \[ 2. Momentum Optimizer 수식 \]
----

* 모멘텀 알고리즘은 Gradient Descent 알고리즘에 지수이동평균 수식을 포함한 수식입니다.
* 모멘텀은 Gradient Descent이 보다 빨리 수렴될 수 있도록 도와줍니다.

## 0. Gradient Descent Optimizer 수식
$$
W := W - \alpha \frac{\partial Loss}{\partial W}
$$

원래 경사하강법의 수식은 위와 같습니다. 이 수식이 어떻게 변형되는지를 한번 보도록 하겠습니다.

<br>

## 1. Momentum Optimizer 수식

$$
V := \beta V +\frac{\partial L}{\partial W} \\
W := W - \alpha V
$$

* 모멘텀 Optimizer의 수식은 지수이동평균 공식을 이용하되, 좀 더 간결하게 표기하였습니다.$(1-\beta)$를 제거하였습니다.

* 모멘텀은 물리에서 관성을 의미합니다. 관성의 힘으로 평탄하지 않은 Loss Function 위를<br>
매끈하게 지나갈 수 있도록 도와줍니다.<br>
* $\beta$에 따라, 관성의 크기를 결정할 수 있습니다. 관성이 클수록, 가속이 빨라지지만<br>
변화에 대처하는 속도가 느려, 수렴이 느려질 수 있습니다. 보통 0.9정도를 이용합니다. <br>

![Imgur](https://i.imgur.com/WHfLZpK.png)

<br>

## 2. NAG Momentum Optimizer 수식

$$
W_{nag} = W - \beta v \\
V := \beta v +\frac{\partial{L}}{\partial{W_{nag}}} \\
W := W - \alpha V
$$

![Imgur](https://i.imgur.com/xSH4hck.jpg)

* NAG 모멘텀 알고리즘은 모멘텀 알고리즘의 수정 본입니다. 차이는 어느 시점의 기울기를 이용할 것인가에 따라 달라집니다. NAG 모멘텀은 momentum에 의해 이동한 상태에서의 gradient을 구한다는 데에서 차이가 있습니다.<br>
* NAG를 이용할 경우, Momentum 방식보다 늘 좀더 효과적인 것으로 알려져 있습니다. NAG 알고리즘이 Momentum보다 훨씬 안정적으로 수렴하기 때문입니다.
* 실제로 Tensorflow와 같은 데에서 어떤 식으로 구현되어 있는가는 [해당 글](https://stackoverflow.com/questions/50774683/how-is-nesterovs-accelerated-gradient-descent-implemented-in-tensorflow)을 찾아보면 됩니다.

<br><br>

# \[ 3. Momentum Optimizer을 시각화하기 \]

----


##  Beale Function Visualization

* 최적화 함수(Optimizer)를 평가하기 위해, 보통 Beale Function과 같은 test function을 많이 이용합니다.

$$
f(w_1,w_2) = (1.5 - w_1 + w_1w_2)^2+(2.25-w_1+w_1w_2^2)^2+(2.625-w_1+w_1w_2^3)^2
$$

In [ ]:
def beale(w1, w2):
    return ((1.5 - w1 + w1 * w2) ** 2 + 
            (2.25 - w1 + w1 * w2 ** 2) ** 2 + 
            (2.625 - w1 + w1 * w2 ** 3) ** 2)

In [ ]:
def generate_beale_plot():
    # Beale function을 공간으로 치환한 것을 
    xmin, xmax, xstep = -4.5, 4.5, .2
    ymin, ymax, ystep = -4.5, 4.5, .2

    w1s = np.arange(xmin, xmax, xstep)
    w2s = np.arange(ymin, ymax, ystep)

    w1, w2 = np.meshgrid(w1s, w2s)
    z = beale(w1, w2)

    minima = np.array([3., .5])
    minima_ = minima.reshape(-1, 1)
    z_minima = beale(*minima)

    plt.figure(figsize=(8, 5))
    ax = plt.axes(projection='3d', elev=50, azim=-50)

    ax.plot(*minima_, z_minima, 'r*', markersize=10)
    ax.plot_surface(w1, w2, z, norm=LogNorm(), rstride=1, cstride=1,
                    edgecolor='None', alpha=0.3, cmap=plt.cm.jet)

    ax.view_init(30, 10)
    ax.set_title("Beale Function Visualization")
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')

    ax.set_xlim((xmin, xmax))
    ax.set_ylim((ymin, ymax))

    return ax

In [ ]:
ax = generate_beale_plot()

## 1. Gradient Descent Optimizer 시각화하기

In [ ]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

In [ ]:
optimizer = SGD(learning_rate = 1e-5)

start_w1, start_w2 = -2., -4.

w1 = K.variable(start_w1)
w2 = K.variable(start_w2)

history = []
for i in tqdm(range(2000)):
    with tf.GradientTape() as tape:
        out = beale(w1, w2)
    history.append((w1.numpy(), w2.numpy(), out.numpy()))
    
    variables = [w1, w2]
    gradients = tape.gradient(out, variables)
    optimizer.apply_gradients(zip(gradients, variables))

In [ ]:
gd_xs, gd_ys, gd_zs = zip(*history)

ax = generate_beale_plot()
ax.plot(gd_xs, gd_ys, gd_zs, 
        label='Gradient Descent', 
        color='b')
plt.show()

## 2. Momentum Optimizer 시각화하기
---

### (1) Momentum이 0.9일 때 시각화

In [ ]:
momentum = 0.9
optimizer = SGD(learning_rate = 1e-5, momentum=momentum)

start_w1, start_w2 = -2., -4.

w1 = K.variable(start_w1)
w2 = K.variable(start_w2)

history = []
for i in tqdm(range(2000)):
    with tf.GradientTape() as tape:
        out = beale(w1, w2)
    history.append((w1.numpy(), w2.numpy(), out.numpy()))
    
    variables = [w1, w2]
    gradients = tape.gradient(out, variables)
    optimizer.apply_gradients(zip(gradients, variables))

In [ ]:
# 경로를 시각화하기
gd_xs, gd_ys, gd_zs = zip(*history)

ax = generate_beale_plot()
ax.plot(gd_xs, gd_ys, gd_zs, 
        label='momentum(beta={})'.format(momentum), 
        color='b')
plt.show()

### (2) Momentum에 따른 변화

In [ ]:
for momentum  in [0.5,0.9,0.99]:
    optimizer = SGD(learning_rate = 1e-5, momentum=momentum)

    start_w1, start_w2 = -2., -4.

    w1 = K.variable(start_w1)
    w2 = K.variable(start_w2)

    history = []
    for i in tqdm(range(2000)):
        with tf.GradientTape() as tape:
            out = beale(w1, w2)
        history.append((w1.numpy(), w2.numpy(), out.numpy()))

        variables = [w1, w2]
        gradients = tape.gradient(out, variables)
        optimizer.apply_gradients(zip(gradients, variables))
        
    # 경로를 시각화하기
    gd_xs, gd_ys, gd_zs = zip(*history)

    ax = generate_beale_plot()
    ax.plot(gd_xs, gd_ys, gd_zs, 
            label='momentum(beta={})'.format(momentum), 
            color='b')
plt.legend()
plt.show()        

## 3. NAG Momentum Optimizer 시각화하기
---

### (1) Momentum이 0.9일 때 시각화

In [ ]:
momentum = 0.9
optimizer = SGD(learning_rate = 1e-5, momentum=momentum, nesterov=True)

start_w1, start_w2 = -2., -4.

w1 = K.variable(start_w1)
w2 = K.variable(start_w2)

history = []
for i in tqdm(range(2000)):
    with tf.GradientTape() as tape:
        out = beale(w1, w2)
    history.append((w1.numpy(), w2.numpy(), out.numpy()))
    
    variables = [w1, w2]
    gradients = tape.gradient(out, variables)
    optimizer.apply_gradients(zip(gradients, variables))

In [ ]:
gd_xs, gd_ys, gd_zs = zip(*history)

ax = generate_beale_plot()
ax.plot(gd_xs, gd_ys, gd_zs,
        label='NAG momentum(beta={})'.format(momentum), 
        color='b')
plt.show()

### (2) Momentum에 따른 변화

In [ ]:
for momentum  in [0.5,0.9,0.99]:
    optimizer = SGD(learning_rate = 1e-5, momentum=momentum,
                    nesterov=True)

    start_w1, start_w2 = -2., -4.

    w1 = K.variable(start_w1)
    w2 = K.variable(start_w2)

    history = []
    for i in tqdm(range(2000)):
        with tf.GradientTape() as tape:
            out = beale(w1, w2)
        history.append((w1.numpy(), w2.numpy(), out.numpy()))

        variables = [w1, w2]
        gradients = tape.gradient(out, variables)
        optimizer.apply_gradients(zip(gradients, variables))
        
    # 경로를 시각화하기
    gd_xs, gd_ys, gd_zs = zip(*history)

    ax = generate_beale_plot()
    ax.plot(gd_xs, gd_ys, gd_zs, 
            label='nag momentum(beta={})'.format(momentum), 
            color='b')
plt.legend()
plt.show()        

#  

---

    Copyright(c) 2019 by Public AI. All rights reserved.<br>
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/04/05

---